<a href="https://colab.research.google.com/github/rida-manzoor/GenAI/blob/main/5_1_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain_community
!pip install -q langchain_objectbox
!pip install -q langchain_openai
!pip install -q langchain_text_splitter
!pip install -q langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 k

In [3]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 7.6 MB/s eta 0:00:00


In [4]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('data.pdf')
docs=loader.load()

In [5]:
docs


[Document(metadata={'source': 'data.pdf', 'page': 0}, page_content='The\nPUCIT\nDepartment\nof\nData\nScience\nwas\nestablished\non\nJanuary\n1st,\n2021.\nThe\nChairman\nof\nthe\nDepartment\nof\nData\nScience\nis\nDR.\nMUHAMMAD\nKAMRAN\nMALIK.\nHe\nhas\ndone\nPhD\nComputer\nScience,\nPU.\nCurrently,\ntwo\nprograms\nare\noffered:\nBS\nData\nScience\nand\nMPhil\nData\nScience.\nFaculty\nincludes\nDr\nAdnan\nAbid,\nDr\nShahid\nManzoor,\nDr\nMuhammad\nNadeem\nMajeed,\nDr\nSyed\nFaisal\nBukhari,\nDr\nKhurram\nShahzad,\nand\nDr\nMuhammad\nArif\nButt.\nDr.\nMuhammad\nArif\nButt\nis\nan\nAssistant\nProfessor\nat\nthe\nDepartment\nof\nData\nScience,\nUniversity\nof\nthe\nPunjab\n(PU),\nLahore,\nPakistan.\nHe\nreceived\nhis\nMSc\nand\nMPhil\ndegrees\nboth\nwith\na\nGold\nMedal\nfrom\nPUCIT,\nUniversity\nof\nthe\nPunjab.\nDr.\nButt\nalso\nearned\nhis\nPh.D.\nin\nComputer\nScience\nfrom\nthe\nsame\nUniversity.\nHis\nresearch\nfocuses\non\napplying\nfuzzy\ninference\nmodels\nin\noperating,\nembedde

## Covert Data into vectorbase

In [6]:
from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain_openai import OpenAIEmbeddings

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [9]:
!pip install -q openai


from google.colab import userdata
from openai import OpenAI
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

In [11]:
from langchain_openai import OpenAIEmbeddings
vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(api_key=userdata.get("OPENAI_API_KEY")), embedding_dimensions=768)
vector

In [12]:
# Making rag pipeline
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [14]:
llm = ChatOpenAI(model="gpt-4o", api_key=userdata.get("OPENAI_API_KEY"))
prompt = hub.pull("rlm/rag-prompt")
prompt

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [15]:
prompt


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [16]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [17]:
question = "who is arif butt?"
result = qa_chain({"query": question })
result

<ipython-input-17-e20883149d51>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question })


{'query': 'who is arif butt?',
 'result': 'Dr. Muhammad Arif Butt is an Assistant Professor at the Department of Data Science, University of the Punjab, Lahore, Pakistan. He has a rich background in academia, research, and administration, with over 32 years of experience, including service in the Pakistan Army. His research focuses on applying fuzzy inference models in systems where decision-making involves imprecise parameters.'}

In [18]:
result["result"]

'Dr. Muhammad Arif Butt is an Assistant Professor at the Department of Data Science, University of the Punjab, Lahore, Pakistan. He has a rich background in academia, research, and administration, with over 32 years of experience, including service in the Pakistan Army. His research focuses on applying fuzzy inference models in systems where decision-making involves imprecise parameters.'